In [1]:
import os
import copy
import numpy as np
import Greedy as gr
import Greedy
import Pixie as px
import Layout
import time
import importlib
import tools
from os import listdir
from os.path import isfile, join

## Emptying procedure 

The Emptying procedure dismantles a stack $s_o$ by taking some elements from it and placing them, preferably ordered, in other stacks. 
$s_o$ simply corresponds to the smallest stack in the current layout (can be ordered or not). If two or more stacks have the same height, then the unordered stack with the highest average group value is selected, otherwise, the ordered stack with the lowest group value is selected.

References to the elements removed from $s_o$ are stored in a list $R$, then, the list is ordered by decreasing group value. This list shows the order in which the elements should be placed back in the stack $s_o$.

In order to assure that an item removed from $s_o$ can be placed back in an orderly way, it is required that the first elements of $R$ be placed into the highest positions of other stacks. Specifically, the tier $t(c)$ of a removed item $c$ in the destination stack should satisfy the following {\it emptying condition}:

$t(c) > H - (\mbox{index}(R,c) + \mbox{slack})$,

where $\mbox{index}(R,c)$ is the position of the item in the list $R$, and \box{slack} corresponds to the total number of empty slots in stacks different from $s_o$ **after the stack being emptied**. The Emptying\& procedure takes into account the emptying condition when removing items from the dismantled stack. 

If, when emptying a stack $s_o$, an item cannot satisfy the emptying condition, then a destination stack $s_d$ for the item is selected and pre-filled with other items until reaching the required height



In [2]:
def Emptying(layout, s_o):
    #Se calculan los ranks
    R = Greedy.get_ranks(layout.stacks[s_o])
    
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    
    fixed_elements = 0

    #eliminar de rank elementos bien colocados
    while len(layout.stacks[s_o]) > 0: #fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d, xg = Layout.select_destination_stack (layout, s_o, max_pos=R.index(top)+1+slack)
        if verbose: print(layout.stacks)
        pos = layout.H - len(layout.stacks[s_d])
        
        if R.index(top)+1 < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            #print()
            if verbose: 
                for s in layout.stacks: print(s)
                print("fill_stack")
                print(s_d,(pos - slack) - R.index(top)-1,s_o,R)
            success = Greedy.fill_stack(layout, s_d, (pos - slack) - (R.index(top)+1), s_o, R)
            if not success: return False, R
            #layout.stacks[s_o].append(c)
        capac -= 1
        c = layout.move(s_o,s_d)
        if xg: R.remove(c)
        if Layout.reachable_height(layout,s_o) >= layout.H: 
            for item in layout.stacks[s_o]: R.remove(item)
            return True, R
        
     #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
    for item in layout.stacks[s_o]: R.remove(item)
    return True, R

## Filling Procedure

The Filling procedure performs a BG move by moving a container from an unordered stack $s_o$ into an ordered stack $s_d$.
The stacks are selected considering the second basic rule, i.e., we attempt to minimize the difference between the group values of the stacks: $\min g(s_d) - g(s_o)$. Note that a BG move also requires that $g(s_d) - g(s_o) \geq 0$. The time complexity of the procedure is linear in the number of stacks.

## Re-Filling Procedure

The Refilling procedure makes use of the basic Filling procedure for selecting the next move to be done. 
The stack $s_o$ prioritizes to place in an orderly way the items from $R$. Four situations may occur:

1. The Filling procedure attempts to place an element in a stack different from $s_o$. 
2. The Filling procedure attempts to place the first element from $R$ in $s_o$.
3. The Filling procedure attempts to place in $s_o$ an element with {\it equal or larger} group value than the first element from $R$. 
4. The Filling procedure attempts to place in $s_o$ an element with {\it smaller} group value than the first element from $R$. 

In the first three situations, the element is placed normally and removed from $R$ (if applicable). In the last situation, the \&Refilling procedure will try to place the **first element** $c$ of $R$ into $s_o$. To do this, 
the items blocking $c$ are placed into stacks different from $s_o$ (auxiliary stacks) and, finally, $c$ is placed into $s_o$ and removed from $R$.


In [3]:
import sys
def ReFilling(layout, s_d, rank):
    ub = Layout.gvalue(layout.stacks[s_d])
    conts = set()

    for r in rank:
        if r>ub: continue
        if r in conts: 
            conts.remove(r)
            continue
        
        while True:
            c = -1
            s, _ = Greedy.SF_move_(layout) #Filling procedure
            s_o = None; s_dd = None
            if s != False: s_o, s_dd = s

            if s != False and s_d == s_dd and layout.stacks[s_o][-1] >= r: 
                c = layout.move(s_o,s_d)
                ub = c
            elif s==False or s_d == s_dd: #force move               
                if verbose==True:
                    if s!=False: print("rank:",r,"  c:",layout.stacks[s_o][-1], "s_o, s_d:",s_o,s_d)
                    else: print("rank:",r)
                    print(layout.stacks)
                ret = Greedy.search_highest(layout, r, ub, s_d) 
                if ret == None: ub = r-1; break
                (s, pos) = ret

                c, state = Greedy.force_move(layout, s, pos, s_d) 
                if state == "failed" and ub == r: print ("NO DEBERIA"); break # NO DEBERIA OCURRIR
                ub = c
            else:
                c = layout.move(s_o, s_dd)

            if len(layout.stacks[s_d]) == layout.H: return True
            #if len(layout.stacks[s_d]) == layout.H-1 and layout.full_stacks >= len(layout.stacks)-3: return True
            
            if c==r: break
            conts.add(c)
    return True
        

In [4]:
def solve(layout, limit=1000):    
        Greedy.lazy_greedy(layout)

        while layout.unsorted_stacks>0 and layout.steps<limit:
            seleccionada = Greedy.select_dismantling_stack(layout)
        
            if verbose:
                for s in layout.stacks: print(s)
                print("\n")

            if verbose: 
                print("selected:",layout.stacks[seleccionada])

            #Se retiran los elementos de la columna
            success, R = Emptying(layout, seleccionada)
            if not success: return 1000

            if verbose:
                for s in layout.stacks: print(s)
                print("\n")


            success = ReFilling(layout,seleccionada, R)
            if not success: return 1000

            Greedy.lazy_greedy(layout)

        return layout.steps

### Solve all the instances

In [5]:
onlyfiles = [f for f in listdir("../Instancias/CVS") if not isfile(join("../Instancias/CVS", f))]
verbose = False
bsg_verbose=False

tot_moves=0
for f in onlyfiles:
    if f == 'BF' or f== '3-x' : continue
    #print(f)
    nmoves=0; count = 0
    av = ["None"]
    start = time.time()
    for i in tools.progressbar(range(1,41), "CVS-"+f+ ":\t", 40, out=av):
        filename = "../Instancias/CVS/"+ f+ "/data"+f+"-"+str(i)+".dat"
        layout = px.read(filename)
        steps = solve(layout)
        #steps = BSG(layout, w=20)
        #print(filename, steps)
        if steps < 1000:
            count += 1
            nmoves += steps
            av[0] = str(nmoves/count)+"\t"+str(time.time() - start) +" "*10
        else: print(filename+"\n")
    tot_moves += nmoves
    #print (f,nmoves/ count,count, " "*50);
    sys.exit
print("prom CVS:",tot_moves/(21.0*40.0))

verbose=False
dir = "../Instancias/BF"
tot_moves=0
carpetas = os.listdir(dir)
#av = ["None"]
for i in range(1,33):
    carpeta = "BF"+str(i)
    archivos = os.listdir(dir+"\\"+carpeta)
    suma = 0
    count = 0
    start = time.time()
    for archivo in archivos:
        if "optimo" not in archivo: 
            ruta = dir+"\\"+carpeta+"\\"+archivo
            #print(ruta)
            layout = px.readBF(ruta)
            steps = solve(layout)
            #steps = BSG(layout, w=20)
            if steps < 1000:
                suma += steps
                count += 1
                #av[0] = str(nmoves/count)+" "+str(time.time() - start) +" "*10
            #suma += gr.greedy_solve(layout)
            
        
    promedio = suma/count  
    tot_moves += promedio
    print(carpeta + ":\t"+str(promedio)+"\t"+str(time.time() - start))
print("prom BF:",tot_moves/(32.0))

CVS-10-10:	163.575	0.642045259475708                                                                                          
CVS-10-6:	126.7	0.532231330871582                                                                                          
CVS-3-3:	11.775	0.4085071086883545                                                                                          
CVS-3-4:	11.875	0.406998872756958                                                                                          
CVS-3-5:	12.575	0.39417600631713867                                                                                          
CVS-3-6:	13.375	0.41103291511535645                                                                                          
CVS-3-7:	14.975	0.4008755683898926                                                                                          
CVS-3-8:	15.5	0.3972814083099365                                                                                          


In [6]:
layout = px.read("../Instancias/CVS/5-4/data5-4-2.dat")

In [10]:
layout.stacks

[[1, 15, 17, 5, 6], [3, 11, 14, 10, 4], [12, 18, 13, 9, 20], [16, 2, 8, 7, 19]]

In [26]:
unsorted_stacks = [ len(layout.stacks[i])-layout.sorted_elements[i] for i in  range(len(layout.stacks)) ] 
n_bx = sum(unsorted_stacks)+min(unsorted_stacks)
n_bx

18

15

In [22]:
layout = px.read("../Instancias/CVS/5-4/data5-4-2.dat")
verbose=False; solve(layout)
moves = layout.moves; inv_moves =[]
layout = px.read("../Instancias/CVS/5-4/data5-4-2.dat")



from tkinter import * 
import tkinter


def inv_op():
    global labelframe1,inv_moves
    labelframe1.grid_remove()
    labelframe1=LabelFrame(top, text="")
    m = inv_moves.pop(0)
    layout.move(m[1],m[0])
    moves.insert(0,m)
    draw_layout(labelframe1,layout)
    return labelframe1

def op():
    global labelframe1,inv_moves
    labelframe1.grid_remove()
    labelframe1=LabelFrame(top, text="")
    m = moves.pop(0)
    layout.move(m[0],m[1])
    inv_moves.insert(0,m)
    draw_layout(labelframe1,layout)
    return labelframe1

#/////////////////////////////////////////    
top = Tk()  
#top.geometry("800x500")  
#Se obtiene tamaño de la pantalla
windowWidth = top.winfo_reqwidth()
windowHeight = top.winfo_reqheight()
positionRight = int(top.winfo_screenwidth()/4 - windowWidth/4)
positionDown = int(top.winfo_screenheight()/4 - windowHeight/4)

top.geometry("+{}+{}".format(positionRight, positionDown))
#Se crean los panels

#BOTONES
labelframe2 = LabelFrame(top, text="")
labelframe2.grid(padx=20, pady=20,columnspan=len(layout.stacks), rowspan=1)

button = tkinter.Button(labelframe2, text="prev",padx=13, command=inv_op)
button.grid(row=0, column=1)

button2 = tkinter.Button(labelframe2, text="next",padx=13, command=op)
button2.grid(row=0, column=6)
#/////////////////////////////////////////////////////////////////

#columnas ahora con colores
labelframe1 = LabelFrame(top, text="")

def draw_layout(labelframe1,layout):
    labelframe1.grid(padx=10, pady=10,columnspan=len(layout.stacks), rowspan=layout.H)



    for i in range(len(layout.stacks)):
        #se obtiene el stack
        s=layout.stacks[i]
        #color
        aux=24500
        cont=1
        for item in s:
            if item <= aux:
                aux=item
                cont+=1
            else:
                break
        cont+=-1
        #Color de la columna
        color=len(s)-cont
        #/////////
        height=layout.H-len(s)
        #print(color)
        j=0
        #Aqui se imprimen los cuadros vacios para que no quede al reves el visual
        while j < height:
            tkinter.Label(labelframe1, text="",borderwidth=10, bg="white", padx=9, pady=1  ).grid(row=j,column=i)
            j+=1

        #Se imprimen en pantalla los valores
        for item in reversed(s):
            if color>0:
                if item > 9:
                    tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=10,pady=10,bg="salmon").grid(row=j,column=i)
                else:
                    tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=13, pady=10,bg="salmon").grid(row=j,column=i)
                color+=-1
            else:
                if item>9:
                    tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=10, pady=10,bg="lime" ).grid(row=j,column=i)
                else:
                    tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=13, pady=10,bg="lime" ).grid(row=j,column=i)
            #veamos si se puede pintar
            j+=1


draw_layout(labelframe1,layout)


top.mainloop()

In [3]:
layout.moves

[(0, 2), (0, 1), (0, 3), (0, 1)]